In [2]:
!pip install pysynth

In [3]:
import pyodbc
import pandas as pd
import numpy as np
import pysynth
import subprocess
import random

In [4]:
from credentials import *

ModuleNotFoundError: No module named 'credentials'

In [89]:
portico = pyodbc.connect("DSN=SIS;UID="+UCL_USERNAME+";PWD="+UCL_PASSWORD)

In [90]:
# real portico data
sql = '''
select cap_ayrc, cap_crsc, stu.stu_gend, stu.stu_codc, cap_idrc
from srs_cap cap left outer join sismgr.ins_stu stu on cap.cap_stuc = stu.stu_code
where cap_ayrc in ('2020', '2019', '2018') and cap_crsc like 'TMSCOM%' and cap_facc='ENG' 
'''
real_data = pd.read_sql(sql, portico)
real_data['STU_CODC'] = real_data['STU_CODC'].astype(str)

In [91]:
len(real_data)

12751

In [92]:
columns = real_data.columns.to_numpy()
columns

array(['CAP_AYRC', 'CAP_CRSC', 'STU_GEND', 'STU_CODC', 'CAP_IDRC'],
      dtype=object)

In [93]:
fake_data = pd.DataFrame(columns=columns)
fake_data.head() 

,CAP_AYRC,CAP_CRSC,STU_GEND,STU_CODC,CAP_IDRC


In [94]:
# fake_data random samples from real data

fake_data = real_data
fake_data['CAP_AYRC'] = np.random.permutation(fake_data['CAP_AYRC'])
fake_data['CAP_CRSC'] = np.random.permutation(fake_data['CAP_CRSC'])
fake_data['STU_GEND'] = np.random.permutation(fake_data['STU_GEND'])
fake_data['STU_CODC'] = np.random.permutation(fake_data['STU_CODC'])
fake_data['CAP_IDRC'] = np.random.permutation(fake_data['CAP_IDRC'])

fake_data.head()

,CAP_AYRC,CAP_CRSC,STU_GEND,STU_CODC,CAP_IDRC
0,2019,TMSCOMSSML01,M,5826,CFUF
1,2019,TMSCOMSSSE01,F,672,C
2,2020,TMSCOMSCGV01,M,764,R
3,2019,TMSCOMSSSE01,M,631,R
4,2019,TMSCOMSING01,M,771,R


In [95]:
fake_data.to_csv('ucl_cs_pg_admissions_synthesized.csv', index=False)

In [97]:
len(fake_data)

12751

In [101]:
# helper files

sql = "select distinct * from srs_cod"
cod = pd.read_sql(sql, portico)
cod = cod[['COD_CODE', 'COD_NAME']]
cod.head()

,COD_CODE,COD_NAME
0,110,East Riding
1,111,North East Lincolnshire
2,112,North Lincolnshire
3,113,York
4,114,Bournemouth


In [102]:
cod[cod['COD_CODE'] == '5826']

,COD_CODE,COD_NAME
313,5826,England


In [103]:
cod.to_csv('ucl_country_of_domicile_codes.csv', index=False)

In [105]:
sql = "select distinct * from srs_crs"
crs = pd.read_sql(sql, portico)
crs = crs[['CRS_CODE', 'CRS_NAME']]
crs.head()

,CRS_CODE,CRS_NAME
0,TAFEDISCLD01,Affil - Cat 2 Clinical Dentistry
1,TAFEDISORT01,Affil - Cat 1 Orthodontics
2,TAFEDISPDD01,Affil - Cat 1 Paediatric Dentistry
3,TAFEENSEEE01,Affil - Cat 1 Electronic & Electrical Engineering
4,TAFEENSMMO01,Affil - Cat 1 Microwaves and Modern Optics


In [106]:
crs.to_csv('ucl_course_codes.csv', index=False)